## 11.2 파이썬은 시퀀스를 찾아낸다.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [2]:
f = Foo()
f[1]

10

In [3]:
for i in f:
    print(i)

0
10
20


In [4]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

In [5]:
from random import shuffle
l = list(range(10))
shuffle(l)

In [8]:
l

[6, 8, 5, 2, 4, 9, 1, 0, 3, 7]

In [9]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

In [11]:
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='4', suit='hearts'),
 Card(rank='J', suit='diamonds'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='J', suit='spades')]

## 11.4 알렉스 마르텔리의 물새

In [15]:
field_names = "Henry James donoban"

try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass

field_names = tuple(field_names)
field_names

('Henry', 'James', 'donoban')

## 11.5 ABC 상속하기

In [16]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    # 카드를 섞기 위해서 setitem 메서드만 있으면 된다. 
    def __setitem__(self, position, value): 
        self._cards[position] = value

    # 그러나 MutableSequence를 상속했으므로, 추상 메서드인 __delitem__()도 구현
    def __delitem__(self, position):  
        del self._cards[position]

    # 세 번째 추상 메서드인 insert()도 구현한다. 
    def insert(self, position, value):  
        self._cards.insert(position, value)

## 11.7 ABC의 정의와 사용

In [20]:
import abc

class Tombola(abc.ABC):  # abc.ABC 상속

    @abc.abstractmethod
    def load(self, iterable):  # 추상 메서드를 데커레이터로 표시
        ...
        
    @abc.abstractmethod
    def pick(self):  
        ...

    def loaded(self):  # 구상 메서드가 들어갈 수 있다. 
        return bool(self.inspect())  # 반드시 ABC에 정의된 인터페이스만 사용한다. 

    def inspect(self):
        items = []
        while True: 
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

In [21]:
class Fake(Tombola):
    def pick(self):
        return 13
    
Fake

__main__.Fake

In [22]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

In [23]:
import random

class BingoCage(Tombola): 

    def __init__(self, items):
        # 암호화에 적합한 무작위 bytes를 생성한다. 
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)  # 초기화 작업

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items) 

    def pick(self):  
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        self.pick()

In [24]:
import random

class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)  # 인수를 이용해 리스트 생성

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))  
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)  

    def loaded(self):  # loaded 메서드 오버라이드
        return bool(self._balls)

    def inspect(self):  # inspect() 오버라이드
        return tuple(self._balls)

In [26]:
from random import randrange

@Tombola.register  # Tombola의 가상 서브클래스로 등로한다. 
class TomboList(list):  # 리스트 상속

    def pick(self):
        if self:  # list가 비어있지 않으면 True 반환
            position = randrange(len(self))
            return self.pop(position) 
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend 

    def loaded(self):
        return bool(self)  # loaded()메서드를 bool() 함수에 위임한다. 

    def inspect(self):

        return tuple(self)

In [27]:
issubclass(TomboList, Tombola)

True

In [28]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [29]:
TomboList.__mro__

(__main__.TomboList, list, object)

## 11.10 오리처럼 행동할 수 있는 거위

In [31]:
class Struggle:
    def __len__(self): return 23
    
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [32]:
issubclass(Struggle, abc.Sized)

True